# CS202: Compiler Construction

## In-class Exercises, Week of 04/17/2023

----

# Select-Instructions

## Question 1

Write an `X86fun` program that's equivalent to the following `Cfun` program:

```
def add1(n):
  add1start:
    tmp_1 = add(n, 1)
    return tmp_1

def main():
  mainstart:
    tmp_2 = add1
    tmp_3 = tmp_2(5)
    print(tmp_3)
    return 0
```

We will do select-instructions for each function def separately, and keep them separately.


```assembly
def add1:
  add1start:
    movq %rdi, #n
    movq #n, #tmp_1
    addq $1, #tmp_1
    movq #tmp_1, %rax
    jmp add1conclusion

def main:
  mainstart:
    --tmp_2 = add1
    leaq add1(%rip), #tmp_2
    --tmp_3 = tmp_2(5)
    movq $5, %rdi
    callq *#tmp_2

    movq %rax, #tmp_3
    movq #tmp_3, %rdi
    callq print_int

    movq $0, %rax
    call mainclonclusion
    

```

## Question 2

Describe the changes to `select-instructions`.

1. Define a new function `si_def` to compile a function definition
2. New/changed cases: Call, Return, and Function assignment
3. Add a pass-global var called `current_function` to keep track of the current function being compiled

New function `si_def`:
1. Set `current_function` to the name of the function being compiled.
2. Call `si_stmts` on the statements in the CFG block
3. For the name+start block, add statements to set up the arguments: one movq for each parameter, from the parameter passing register to the variable with the parameter name.
4. Return a new x86 function definition

New cases in `si_stmt`:
1. Return should now jump to current_function + "conclusion"
2. `cfun.Assign(x, cfun.Var(f)) if f in `functions_names`: use `x86.NamedInstr('leaq', [x86.GlobalVal(f), x86.Var(x)])` to set up the function pointer
3. `cfun.Assign(x, cfun.Call(f, args))`: 

    1. Move the arguments into the parameter registers (reverse of the definition case)
    2. Perform an indirect callq: `x86.IndirectCallq(si_atm(fun), 0)
    3. Move the return value into the destination variable: `x86.NamedInstr('movq', [x86.Reg('rax'), x86.Var(x)])`
    

----
# Register allocation

## Question 3

Describe the changes to the register allocator.

Copy the pass from A6, name it `_allocate_registers`, and call each pf the functions in the program.

This means we perform register allocation for each function separately.

In addition, change the register allocator to ensure that we follow the calling convention.
- We want to ensure that no value in a caller-saved register will be live across a call to any function.
- Alternative would be to explicitly save the caller-saved registers before doing the call. But this requires more code, and is more complicated to implement.

Change to `bi_instr` in the allocator:
- For call and indirectcallq: add an edge in the interference graph between each live variable and each caller-saved register.

To ensure we don't overwrite parameter-passing registers before we read their values:

- Change `vars_arg` to return registers
- This ensures that we don't overwrite a parameter passing register until after we've used the value that was in.
- Effect is to put registers whose values we still need in the live-after sets of instructions where they might be overwritten.

----
# Patch Instructions

## Question 4

Describe the changes to patch-instructions.

Copy the pass from A6, name it `_patch_instructions`, and call each pf the functions in the program.

----
# Prelude & Conclusion

## Question 5

Describe the changes to prelude & conclusion.

Copy the pass from A6, name it `_prelude_conclusion`, and call each pf the functions in the program.

Also need to flatten out the function definitions into a single x86 program.

Also need to make several other changes.

1. Save and restore callee-saved registers: add pushq instructions to prelude to save tge registers, popq instructions to clonclusion to restore them.

2. Initialize the heap only in the main function: call the initialize function only if the name is main.

3. Add an instruction to tear down the root stack frame
4. use the functions's name as the prelude label
5. use name+conclusion as the conclusion label


----
# Dataclasses and Objects

## Question 6

Transform the following program into an `Ltup` program using tuples.

```
class Point:
    x: int
    y: int

p = Point(1, 2)
v = p.x + p.y
print(v)
```

In [1]:
p = (1,2)
v = p[0] + p[1]
print(v)

3


## Question 7

Describe the changes to the compiler to support dataclasses.

High level idea
1. Represent objects using tuples allocated on the heap
2. Build a mapping from field names to where they are stored in the tuple representing the object itself
3. Turn object constructors into a tuple construction
4. Turn field reference into tuple subscripts, using #2 to know what index to look at.

Detailed changes to the compiler:
1. Modify the typechecker
  - Define a new type for dataclasses
  - Add a dataclase case to tc_stmt that adds a dict type for the object constructor to the type environment
  - Add a field reference case to tc_exp that:
    - Ensures that object is a dataclass type
    - Returns the type of the referenced fields
  - For each dataclass-valued var , remember its type
  - For each dataclass definition, remember its name
2. Update rco
  - Add a case for field reference: call rco_exp on the object to make sure it's atomic
3. Add a new pass right after the second typechecker
  - translate calls to object constructors to calls to tuple
  - translate field references to tuple subscripts
    - get the type of the object (look it up in the saved types from the typechecker)
    - find the index of the referenced field in the type's dict
    - return a tuple subscript expression where the tuple is the object, and the index is the field index

## Question 8

Transform the following program into an `Lfun` program using tuples.

```
class Point:
    x: int
    y: int
    
    def add(self: Point, other: Point) -> Point:
        return Point(self.x + other.x, self.y + other.y)

p1 = Point(1, 2)
p2 = Point(3, 4)
p3 = p1.add(p2)
v = p3.x + p3.y
print(v)
```

In [4]:

def add(self, other):
    return ((add,), self[1] + other[2], self[2] + other[2])

p1 = ((add,),1,2)
p2 = ((add,),3,4)
p3 = p1[0][0](p1,p2)
v = p3[1] + p3[2]
print(v)

11
